# Nacional

In [ ]:
import ee
import pandas as pd

# Trigger the authentication flow.
ee.Authenticate()

# Inicializar Earth Engine
ee.Initialize()

In [ ]:
# Definir la región de interés (Colombia)
colombia = ee.Geometry.Polygon(
    [[[-79.0250, 13.1824], [-79.0250, 4.2279], [-66.8719, 4.2279], [-66.8719, 13.1824]]])

# Cargar la colección de datos MODIS LST
collection = ee.ImageCollection('MODIS/006/MOD11A2')\
    .filterDate('2007-01-01', '2021-12-31')\
    .select(['LST_Day_1km'])\
    .filterBounds(colombia)


# Cargar la colección de datos MODIS de precipitaciones
# chirps_collection = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
#     .filterDate('2007-01-01', '2021-12-31')\
#     .select(['precipitation'])\
#     .filterBounds(colombia)

# Función para calcular la temperatura promedio y las precipitaciones promedio
def reduce_image(image):
    date = ee.Date(image.get('system:time_start'))
    temperature_mean = image.select('LST_Day_1km').reduceRegion(reducer=ee.Reducer.mean(), geometry=colombia, scale=1000)
    temperature_min = image.select('LST_Day_1km').reduceRegion(reducer=ee.Reducer.min(), geometry=colombia, scale=1000)
    temperature_max = image.select('LST_Day_1km').reduceRegion(reducer=ee.Reducer.max(), geometry=colombia, scale=1000)
    #precipitation_mean = chirps_collection.filterDate(date, date.advance(1, 'day')).sum().reduceRegion(reducer=ee.Reducer.mean(), geometry=colombia, scale=1000)
    # precipitation_min = precip_collection.filterDate(date, date.advance(1, 'day')).min().reduceRegion(reducer=ee.Reducer.min(), geometry=colombia, scale=1000)
    # precipitation_max = precip_collection.filterDate(date, date.advance(1, 'day')).max().reduceRegion(reducer=ee.Reducer.max(), geometry=colombia, scale=1000)
    return ee.Feature(None, {'id': date.format('YYYY-MM-dd'),
                             'mean_temperature': temperature_mean.get('LST_Day_1km'),
                             'min_temperature': temperature_min.get('LST_Day_1km'),
                             'max_temperature': temperature_max.get('LST_Day_1km')
                            #  'mean_precipitation': precipitation_mean.get('precipitation'),
                            #  'min_precipitation': precipitation_min.get('precipitation'),
                            #  'max_precipitation': precipitation_max.get('precipitation')
                             })

# def reduce_image1(image):
#     date = ee.Date(image.get('system:time_start'))
#     precipitation_mean = image.select('precipitation').reduceRegion(reducer=ee.Reducer.mean(), geometry=colombia, scale=1000)
#     # precipitation_min = precip_collection.filterDate(date, date.advance(1, 'day')).min().reduceRegion(reducer=ee.Reducer.min(), geometry=colombia, scale=1000)
#     # precipitation_max = precip_collection.filterDate(date, date.advance(1, 'day')).max().reduceRegion(reducer=ee.Reducer.max(), geometry=colombia, scale=1000)
#     return ee.Feature(None, {'id': date.format('YYYY-MM-dd'),
#                             #  'mean_temperature': temperature_mean.get('LST_Day_1km'),
#                             #  'min_temperature': temperature_min.get('LST_Day_1km'),
#                             #  'max_temperature': temperature_max.get('LST_Day_1km'),
#                              'mean_precipitation': precipitation_mean.get('precipitation'),
#                             #  'min_precipitation': precipitation_min.get('precipitation'),
#                             #  'max_precipitation': precipitation_max.get('precipitation')
#                              })

time_series = collection.map(reduce_image)
# time_series1 = chirps_collection.map(reduce_image1)

# Convertir la serie de tiempo a un DataFrame de Pandas
df = pd.DataFrame([feature.toDictionary().getInfo() for feature in time_series.getInfo()['features']])
df = df.set_index('id')
df.index = pd.to_datetime(df.index)

# df1 = pd.DataFrame([feature.toDictionary().getInfo() for feature in time_series1.getInfo()['features']])
# df1 = df1.set_index('id')
# df1.index = pd.to_datetime(df1.index)

print(df)

df.to_csv('climate.csv')

In [ ]:
import pandas as pd

# Definir la región de interés (Colombia)
colombia = ee.Geometry.Polygon(
    [[[-79.0250, 13.1824], [-79.0250, 4.2279], [-66.8719, 4.2279], [-66.8719, 13.1824]]])

# Cargar la colección de datos MODIS LST
collection = ee.ImageCollection('MODIS/006/MOD11A2')\
    .filterDate('2007-01-07', '2021-12-31')\
    .select('LST_Day_1km')\
    .filterBounds(colombia)\
    .filter(ee.Filter.calendarRange(0, 365, 'day_of_year'))

# collection = ee.ImageCollection('MODIS/006/MOD11A2')
# filtered_collection = collection.filterBounds(colombia).filterDate('2007-01-07', '2021-12-31').select('LST_Day_1km').filter(ee.Filter.calendarRange(0, 365, 'day_of_year'))

# Reducir la colección a una serie de tiempo mensual
def reduce_image(image):
    date = ee.Date(image.get('system:time_start'))
    mean_temp = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=colombia, scale=1000)
    mean_temp_value = mean_temp.get('LST_Day_1km')
    return ee.Feature(None, {'id': date.format('YYYY-MM-dd'), 'mean_temperature': mean_temp_value})

time_series = collection.map(reduce_image)

# Convertir la serie de tiempo a un DataFrame de Pandas
df = pd.DataFrame([feature['properties'] for feature in time_series.getInfo()['features']])
df = df.set_index('id')
df.index = pd.to_datetime(df.index)
df['mean_temperature'] = (df['mean_temperature']*0.02)-273.15

print(df)

# Departamentos

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
import pandas as pd
import geopandas as gpd

# Función para convertir tuplas en listas
def convertir_a_lista(tupla):
    return list(tupla)

gdf = gpd.read_file('depto_project.shp')
geometrias = []
for i in range(len(gdf)):
    geometry = gdf.iloc[i]['geometry']
    coords = [convertir_a_lista(coord) for coord in geometry.envelope.exterior.coords]
    geometrias.append(coords)


In [ ]:
dfs = []
for i in range(len(gdf)):

    # Definir la región de interés (Colombia)
    coordenadas_multipolygon = geometrias[i]

    geometria = ee.Geometry.Polygon([coordenadas_multipolygon])#[[[-79.0250, 13.1824], [-79.0250, 4.2279], [-66.8719, 4.2279], [-66.8719, 13.1824]]])
    # geometria = geometria.transform('MODIS/006/MODIS sinusoidal')

    # Cargar la colección de datos MODIS LST
    collection = ee.ImageCollection('MODIS/006/MOD11A2')\
        .filterDate('2007-01-01', '2021-12-31')\
        .select('LST_Day_1km')\
        .filterBounds(geometria)

    # Reducir la colección a una serie de tiempo mensual
    def reduce_image(image):
        date = ee.Date(image.get('system:time_start'))
        mean_temp = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=geometria, scale=1000)
        mean_temp_value = mean_temp.get('LST_Day_1km')
        max_temp = image.reduceRegion(reducer=ee.Reducer.max(), geometry=geometria, scale=1000)
        max_temp_value = max_temp.get('LST_Day_1km')
        min_temp = image.reduceRegion(reducer=ee.Reducer.min(), geometry=geometria, scale=1000)
        min_temp_value = min_temp.get('LST_Day_1km')
        return ee.Feature(None, {'id': date.format('YYYY-MM-dd'), 'mean_temperature': mean_temp_value, 'max_temperature': max_temp_value, 'min_temperature': min_temp_value})

    time_series = collection.map(reduce_image).reduce(ee.Reducer.toList(52))

    # Convertir la serie de tiempo a un DataFrame de Pandas
    df = pd.DataFrame([feature['properties'] for feature in time_series.getInfo()['features']])
    df = df.set_index('id')
    df.index = pd.to_datetime(df.index)
    df['mean_temperature'] = (df['mean_temperature']*0.02)-273.15
    df['max_temperature'] = (df['max_temperature']*0.02)-273.15
    df['min_temperature'] = (df['min_temperature']*0.02)-273.15
    df['depto'] = int(gdf.iloc[i]['DPTO'])
    dfs.append(df.reset_index())
    print(gdf.iloc[i]['NOMBRE_DPT'])
dataset = pd.concat(dfs, ignore_index=True)
print(dataset)

In [ ]:
dataset.to_csv('climate_dptos.csv', index = False)

In [ ]:
# Lluvias
dfs = []
for i in range(len(gdf)):

    # Definir la región de interés (Colombia)
    coordenadas_multipolygon = geometrias[i]

    geometria = ee.Geometry.Polygon([coordenadas_multipolygon])

    # Cargar la colección de datos MODIS LST
    collection = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')\
        .filterDate('2007-01-01', '2021-12-31')\
        .select('precipitation')\
        .filterBounds(geometria)

    # Reducir la colección a una serie de tiempo mensual
    def reduce_image(image):
        date = ee.Date(image.get('system:time_start'))
        mean_rain = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=geometria, scale=1000)
        mean_rain_value = mean_rain.get('precipitation')
        return ee.Feature(None, {'id': date.format('YYYY-MM-dd'), 'mean_rain': mean_rain_value})

    time_series = collection.map(reduce_image)

    # Convertir la serie de tiempo a un DataFrame de Pandas
    df = pd.DataFrame([feature['properties'] for feature in time_series.getInfo()['features']])
    df = df.set_index('id')
    df.index = pd.to_datetime(df.index)
    df['depto'] = int(gdf.iloc[i]['DPTO'])
    dfs.append(df.reset_index())
    print(gdf.iloc[i]['NOMBRE_DPT'])
dataset = pd.concat(dfs, ignore_index=True)
print(dataset)

In [ ]:
dataset.to_csv('rain_dptos.csv', index = False)

# Municipios

In [5]:
!pip install geopandas

  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/27/27/2687abaa2ac02b5814e2929a5033da1e5d132c19a904dc56f77f63fd6eb9/geopandas-0.14.0-py3-none-any.whl.metadata
  Obtaining dependency information for fiona>=1.8.21 from https://files.pythonhosted.org/packages/b0/7f/2de46a2630f609b7520d74ffc7692d4969b1fa1dd3c82f62c7967183d365/Fiona-1.9.4.post1-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/50.6 kB ? eta -:--:--
     -------------------------------- ------- 41.0/50.6 kB 1.9 MB/s eta 0:00:01
     -------------------------------- ------- 41.0/50.6 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 50.6/50.6 kB 367.3 kB/s eta 0:00:00
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/c8/5a/215a1894e50167d91b471d8fc413ca30034c48e5d3dfac78d12df4c840d5/pyproj-3.6.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/1.4

In [3]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AfJohXmqkBETZIWEaJG86r8jP8eObF6rtSnZYqDkyEIVsnqMYLWY3gIfrC4

Successfully saved authorization token.


In [4]:
import pandas as pd
import geopandas as gpd

# Función para convertir tuplas en listas
def convertir_a_lista(tupla):
    return list(tupla)

gdf = gpd.read_file('../Data/SHP_MGN2018_INTGRD_MPIO/MGN_ANM_MPIOS.shp')
gdf

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
cities =['63001',
'08001',
'11001',
'68001',
'76001',
'13001',
'54001',
'18001',
'73001',
'17001',
'05001',
'23001',
'41001',
'52001',
'66001',
'19001',
'27001',
'44001',
'47001',
'70001',
'15001',
'20001',
'50001']

In [ ]:
gdf = gdf[['MPIO_CDPMP', 'MPIO_CNMBR', 'geometry']]
gdf = gdf[gdf['MPIO_CDPMP'].isin(cities)]

In [ ]:
geometrias = []
for i in range(len(gdf)):
    geometry = gdf.iloc[i]['geometry']
    coords = [convertir_a_lista(coord) for coord in geometry.envelope.exterior.coords]
    geometrias.append(coords)

In [ ]:
dfs = []
for i in range(len(geometrias)):

    # Definir la región de interés (Colombia)
    coordenadas_multipolygon = geometrias[i]

    geometria = ee.Geometry.Polygon([coordenadas_multipolygon])#[[[-79.0250, 13.1824], [-79.0250, 4.2279], [-66.8719, 4.2279], [-66.8719, 13.1824]]])
    # geometria = geometria.transform('MODIS/006/MODIS sinusoidal')

    # Cargar la colección de datos MODIS LST
    collection = ee.ImageCollection('MODIS/006/MOD11A2')\
        .filterDate('2007-01-01', '2021-12-31')\
        .select('LST_Day_1km')\
        .filterBounds(geometria)

    # Reducir la colección a una serie de tiempo mensual
    def reduce_image(image):
        date = ee.Date(image.get('system:time_start'))
        mean_temp = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=geometria, scale=1000)
        mean_temp_value = mean_temp.get('LST_Day_1km')
        max_temp = image.reduceRegion(reducer=ee.Reducer.max(), geometry=geometria, scale=1000)
        max_temp_value = max_temp.get('LST_Day_1km')
        min_temp = image.reduceRegion(reducer=ee.Reducer.min(), geometry=geometria, scale=1000)
        min_temp_value = min_temp.get('LST_Day_1km')
        return ee.Feature(None, {'id': date.format('YYYY-MM-dd'),
                                 'mean_temperature': mean_temp_value,
                                 'max_temperature': max_temp_value,
                                 'min_temperature': min_temp_value})

    time_series = collection.map(reduce_image)#.reduce(ee.Reducer.toList(52))

    # Convertir la serie de tiempo a un DataFrame de Pandas
    df = pd.DataFrame([feature['properties'] for feature in time_series.getInfo()['features']])
    df = df.set_index('id')
    df.index = pd.to_datetime(df.index)
    df['mean_temperature'] = (df['mean_temperature']*0.02)-273.15
    df['max_temperature'] = (df['max_temperature']*0.02)-273.15
    df['min_temperature'] = (df['min_temperature']*0.02)-273.15
    df['mpio'] = int(gdf.iloc[i]['MPIO_CDPMP'])
    dfs.append(df.reset_index())
    print(gdf.iloc[i]['MPIO_CNMBR'])
dataset = pd.concat(dfs, ignore_index=True)
print(dataset)

In [ ]:
dataset.to_csv('climate_mpios.csv', index = False)

In [ ]:
# Lluvias
dfs = []
for i in range(len(gdf)):

    # Definir la región de interés (Colombia)
    coordenadas_multipolygon = geometrias[i]

    geometria = ee.Geometry.Polygon([coordenadas_multipolygon])

    # Cargar la colección de datos MODIS LST
    collection = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')\
        .filterDate('2007-01-01', '2021-12-31')\
        .select('precipitation')\
        .filterBounds(geometria)

    # Reducir la colección a una serie de tiempo mensual
    def reduce_image(image):
        date = ee.Date(image.get('system:time_start'))
        mean_rain = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=geometria, scale=1000)
        mean_rain_value = mean_rain.get('precipitation')
        return ee.Feature(None, {'id': date.format('YYYY-MM-dd'), 'mean_rain': mean_rain_value})

    time_series = collection.map(reduce_image)

    # Convertir la serie de tiempo a un DataFrame de Pandas
    df = pd.DataFrame([feature['properties'] for feature in time_series.getInfo()['features']])
    df = df.set_index('id')
    df.index = pd.to_datetime(df.index)
    df['mpio'] = int(gdf.iloc[i]['MPIO_CDPMP'])
    dfs.append(df.reset_index())
    print(gdf.iloc[i]['MPIO_CNMBR'])
dataset = pd.concat(dfs, ignore_index=True)
print(dataset)

In [ ]:
dataset.to_csv('rain_mpios.csv', index = False)

# Datos Mauro

In [12]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AfJohXnOTZkHJ975Inikt0k4YFQgIbw2g3tkFr3gdEp_Y1Nkxhw-0UvJ5YQ

Successfully saved authorization token.


In [13]:
import pandas as pd
import geopandas as gpd

# Función para convertir tuplas en listas
def convertir_a_lista(tupla):
    return list(tupla)

gdf = gpd.read_file('../Data/SHP_MGN2018_INTGRD_MPIO/MGN_ANM_MPIOS.shp')
gdf

,DPTO_CCDGO,MPIO_CCDGO,MPIO_CNMBR,MPIO_CDPMP,VERSION,AREA,LATITUD,LONGITUD,STCTNENCUE,STP3_1_SI,...,STP34_9_ED,STP51_PRIM,STP51_SECU,STP51_SUPE,STP51_POST,STP51_13_E,STP51_99_E,Shape_Leng,Shape_Area,geometry
0,18,001,FLORENCIA,18001,2018,2.547638e+09,1.749139,-75.558239,71877.0,32.0,...,2244.0,48848.0,59610.0,21898.0,4592.0,5892.0,3799.0,2.942508,0.206928,"POLYGON ((-75.42074 2.19413, -75.42042 2.19412..."
1,18,029,ALBANIA,18029,2018,4.141221e+08,1.227865,-75.882327,2825.0,24.0,...,92.0,1940.0,1712.0,231.0,41.0,215.0,46.0,1.112829,0.033618,"POLYGON ((-75.89506 1.36570, -75.89506 1.36570..."
2,18,094,BELÃN DE LOS ANDAQUÃES,18094,2018,1.191619e+09,1.500923,-75.875645,4243.0,54.0,...,145.0,3541.0,3340.0,490.0,119.0,720.0,123.0,2.234657,0.096745,"POLYGON ((-75.78705 1.74982, -75.78715 1.74956..."
3,18,247,EL DONCELLO,18247,2018,1.106076e+09,1.791386,-75.193944,8809.0,0.0,...,348.0,7571.0,6287.0,1029.0,228.0,1095.0,171.0,3.154370,0.089867,"POLYGON ((-75.36167 2.32143, -75.36142 2.32122..."
4,18,256,EL PAUJÃL,18256,2018,1.234734e+09,1.617746,-75.234043,5795.0,0.0,...,248.0,6072.0,4066.0,639.0,108.0,916.0,99.0,3.529316,0.100309,"POLYGON ((-75.36638 2.20952, -75.35963 2.17388..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,25,290,FUSAGASUGÃ,25290,2018,1.939529e+08,4.323534,-74.388591,78073.0,0.0,...,5838.0,39588.0,52504.0,25227.0,3591.0,3823.0,2293.0,1.026143,0.015800,"POLYGON ((-74.29700 4.42407, -74.29696 4.42384..."
1118,25,662,SAN JUAN DE RIOSECO,25662,2018,3.140873e+08,4.841098,-74.658493,5654.0,0.0,...,380.0,3503.0,2385.0,522.0,83.0,588.0,64.0,0.944174,0.025603,"POLYGON ((-74.72826 4.96905, -74.72815 4.96899..."
1119,73,349,HONDA,73349,2018,3.048869e+08,5.179417,-74.783031,11333.0,0.0,...,846.0,7574.0,9888.0,2873.0,311.0,1157.0,457.0,1.064832,0.024864,"POLYGON ((-74.82173 5.31671, -74.82156 5.31653..."
1120,05,631,SABANETA,05631,2018,1.583532e+07,6.138305,-75.608806,37843.0,0.0,...,1521.0,12872.0,27854.0,28441.0,6577.0,1124.0,1035.0,0.172905,0.001293,"POLYGON ((-75.60355 6.15510, -75.60350 6.15506..."


In [14]:
geometrias = []
for i in range(len(gdf)):
    geometry = gdf.iloc[i]['geometry']
    coords = [convertir_a_lista(coord) for coord in geometry.envelope.exterior.coords]
    geometrias.append(coords)

In [15]:
images = [#'SPEI_01_month',
# 'SPEI_02_month',
# 'SPEI_03_month',
# 'SPEI_04_month',
# 'SPEI_05_month',
# 'SPEI_06_month',
# 'SPEI_07_month',
# 'SPEI_08_month',
# 'SPEI_09_month',
# 'SPEI_10_month',
# 'SPEI_11_month',
# 'SPEI_12_month',
# 'SPEI_13_month',
# 'SPEI_14_month',
# 'SPEI_15_month',
# 'SPEI_16_month',
'SPEI_17_month',
'SPEI_18_month',
'SPEI_19_month',
'SPEI_20_month',
'SPEI_21_month',
'SPEI_22_month',
'SPEI_23_month',
'SPEI_24_month',
'SPEI_25_month',
'SPEI_26_month',
'SPEI_27_month',
'SPEI_28_month',
'SPEI_29_month',
'SPEI_30_month',
'SPEI_31_month',
'SPEI_32_month',
'SPEI_33_month',
'SPEI_34_month',
'SPEI_35_month',
'SPEI_36_month',
'SPEI_37_month',
'SPEI_38_month',
'SPEI_39_month',
'SPEI_40_month',
'SPEI_41_month',
'SPEI_42_month',
'SPEI_43_month',
'SPEI_44_month',
'SPEI_45_month',
'SPEI_46_month',
'SPEI_47_month',
'SPEI_48_month']

In [16]:
from tqdm import tqdm
dataset = []
for k in images:
    dfs = []
    for i in tqdm(range(len(gdf))):

        # Definir la región de interés (Colombia)
        coordenadas_multipolygon = geometrias[i]

        geometria = ee.Geometry.Polygon([coordenadas_multipolygon])

        # Cargar la colección de datos MODIS LST
        collection = ee.ImageCollection('CSIC/SPEI/2_8')\
            .filterDate('2006-01-01', '2021-01-01')\
            .select(k)\
            .filterBounds(geometria)

        # Reducir la colección a una serie de tiempo mensual
        def reduce_image(image):
            date = ee.Date(image.get('system:time_start'))
            mean_rain = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=geometria, scale=1000)
            mean_rain_value = mean_rain.get(k)
            max_rain = image.reduceRegion(reducer=ee.Reducer.max(), geometry=geometria, scale=1000)
            max_rain_value = max_rain.get(k)
            min_rain = image.reduceRegion(reducer=ee.Reducer.min(), geometry=geometria, scale=1000)
            min_rain_value = min_rain.get(k)
            median_rain = image.reduceRegion(reducer=ee.Reducer.median(), geometry=geometria, scale=1000)
            median_rain_value = median_rain.get(k)
            std_rain = image.reduceRegion(reducer=ee.Reducer.median(), geometry=geometria, scale=1000)
            std_rain_value = std_rain.get(k)
            return ee.Feature(None, {'id': date.format('YYYY-MM-dd'),
                                     'mean_rain': mean_rain_value,
                                     'max_rain': max_rain_value,
                                     'min_rain': min_rain_value,
                                     'median_rain': median_rain_value,
                                     'std_rain': std_rain_value})

        time_series = collection.map(reduce_image)

        # Convertir la serie de tiempo a un DataFrame de Pandas
        df = pd.DataFrame([feature['properties'] for feature in time_series.getInfo()['features']])
        df = df.set_index('id')
        df.index = pd.to_datetime(df.index)
        df['mpio'] = int(gdf.iloc[i]['MPIO_CDPMP'])
        df['image'] = k
        dfs.append(df.reset_index())
#         print(gdf.iloc[i]['MPIO_CNMBR'])
    dataset.append(pd.concat(dfs, ignore_index=True))
    print(k)
dataset = pd.concat(dataset, ignore_index = True)
print(dataset)

100%|██████████████████████████████████████████████████████████████████████████████| 1122/1122 [23:50<00:00,  1.27s/it]


SPEI_17_month


100%|██████████████████████████████████████████████████████████████████████████████| 1122/1122 [37:41<00:00,  2.02s/it]


SPEI_18_month


 20%|███████████████▉                                                               | 227/1122 [10:46<42:30,  2.85s/it]


EEException: Earth Engine memory capacity exceeded.

In [9]:
dataset1 = pd.concat(dataset[:16], ignore_index = True)
print(dataset1)

                id  max_rain mean_rain median_rain  min_rain  std_rain   mpio  \
0       2006-01-01  1.590711  1.528799    1.516858  1.372851  1.516858  18001   
1       2006-02-01  0.102807  0.012154    0.053621 -0.124127  0.053621  18001   
2       2006-03-01  2.214705  2.095264    2.109531  1.795515  2.109531  18001   
3       2006-04-01  1.280336  1.227882     1.21641  1.155182   1.21641  18001   
4       2006-05-01  0.603359 -0.195452   -0.251063 -1.006755 -0.251063  18001   
...            ...       ...       ...         ...       ...       ...    ...   
3231355 2020-08-01 -0.053438 -0.053438   -0.053438 -0.053438 -0.053438   5380   
3231356 2020-09-01 -0.180793 -0.180793   -0.180793 -0.180793 -0.180793   5380   
3231357 2020-10-01  -0.37478  -0.37478    -0.37478  -0.37478  -0.37478   5380   
3231358 2020-11-01 -0.233254 -0.233254   -0.233254 -0.233254 -0.233254   5380   
3231359 2020-12-01  0.144974  0.144974    0.144974  0.144974  0.144974   5380   

                 image  
0 

In [11]:
dataset1.to_csv('../Data/SPEI_Mauro_1_16.csv', index = True)